In [18]:
from datetime import datetime, timedelta, time
kst_now = datetime.utcnow() + timedelta(hours = 9)
edt_now = datetime.utcnow() - timedelta(hours = 4)
kst_weekno = kst_now.weekday()
edt_weekno = edt_now.weekday()
print(kst_weekno, edt_weekno)

1 0


In [15]:
datetime.utcnow() + timedelta(hours = 9)

datetime.datetime(2022, 6, 28, 8, 41, 19, 41399)

In [13]:
(datetime.utcnow() - timedelta(hours = 4)).weekday()

0

In [14]:
datetime.utcnow() - timedelta(hours = 4)

datetime.datetime(2022, 6, 27, 19, 41, 9, 631005)

In [5]:
now.time()

datetime.time(9, 18, 51, 996802)

In [6]:
now.time() > time(hour=5, minute=29)

True

In [28]:
future = datetime.combine((datetime.today() + timedelta(days=2)), datetime.strptime("00:00", "%H:%M").time())
(future - now).total_seconds()

137632.769919

In [1]:

import os
import sys
from os import path

import logging
import pause

import threading
from threading import Timer, Thread
import schedule

import pandas as pd
import numpy as np

from datetime import datetime, timedelta, time
import time as ot

import requests
from public_data_config import apisdata



from slack_message import _post_message
from slack_sdk_message_post import _post_message_with_slack_sdk

# Import WebClient from Python SDK
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

from get_holiday import _get_holiday
from make_stock_list import _get_stock_list
from make_cci_signal import _make_cci_signal
from collect_signal import _collect_cci_signal

from slack_message import _post_message
from slack_sdk_message_post import _post_message_with_slack_sdk

In [5]:
class BaseService:
    def __init__(self):
        self.myToken = apisdata["slack"]["token"]
        self.channel_name = apisdata["slack"]["channel_name"]
        self.channel_id = apisdata["slack"]["channel_id"]

        # 휴장일 데이터 가져오기
        self.mykey = apisdata["holiday"]["decoding_key"]
        self.request_url = apisdata["holiday"]["request_url"]
        self.year = "2022"
        self.market = "kospi"
        self.market_stock_dict = {
            self.market : None,
            }
        self.market_dict = None
        self.config_ndays = 20
        self.financial_engineering_days = 252
        self.start_date = (datetime.now() - timedelta(days=self.financial_engineering_days)).date().strftime("%Y-%m-%d")
        self.ko_holiday_list =  ['2022-01_01','2022-01_31','2022-02_01','2022-02_02','2022-03_01','2022-03_09','2022-05_05','2022-05_08','2022-06_01','2022-06_06','2022-08_15','2022-09_09','2022-09_10','2022-09_11','2022-09_12','2022-10_03','2022-10_09','2022-10_10','2022-12_25']

    def scheduler(self):
        today_date = datetime.now().date().strftime("%Y-%m-%d")
        try:
            holiday_list = _get_holiday(self)
        except Exception as e:
            print(e)
            holiday_list = []

        if today_date not in holiday_list:
            try:
                self._get_stock_list()

                self._post_signal_to_slack()
            except Exception as e:
                self._post_message(e)
                pass

        else:
            try:
                self._get_stock_list()
                self._post_signal_to_slack()
            except Exception as e:
                self._post_message(e)
                pass
    
    def _get_stock_list(self):
        stock_list = []
        if self.market == "kospi":
            for i in range(1, 21):
                page = i
                url = 'https://finance.naver.com/sise/entryJongmok.nhn?&page={page}'.format(page = page)
                source = urlopen(url).read()
                source = BeautifulSoup(source,'lxml')
                source = source.find_all('a',target = '_parent')
                for j in range(len(source)):
                    name = source[j].text
                    stock_list.append(name)

            kospi_df = fdr.StockListing(self.market)
            self.market_dict = kospi_df[kospi_df["Name"].isin(stock_list)].set_index("Symbol")["Name"].to_dict()
            self.market_stock_dict[self.market] = self.market_dict

        elif self.market == "sp500":
            self.market_dict = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].set_index("Symbol")["Security"].to_dict()
            self.market_stock_dict[self.market] = self.market_dict


    def _post_message(self, text):
        response = requests.post("https://slack.com/api/chat.postMessage",
            headers={"Authorization": "Bearer "+self.myToken},
            data={"channel": self.channel_name,"text": text}
        )
        if response.ok:
            print("Success. Send message with slack.")
        else:
            print("Fail. Send message with slack.")
            pass


    def _post_message_with_slack_sdk(self, text=None, blocks=None):
        client = WebClient(token=self.myToken)
        logger = logging.getLogger(__name__)
        channel_id = self.channel_id

        try:
            # Call the conversations. list method using the WebClient
            if text is not None:
                result = client.chat_postMessage(
                    channel=channel_id,
                    text = text
                )
                print(result)

            elif blocks is not None:
                result = client.chat_postMessage(
                    channel=channel_id,
                    blocks = blocks
                )
                print(result)
        except SlackApiError as e:
            print(f"Error: {e}")

    def _make_signal(self, df):
        try:
            df = df.reset_index()
            df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3
            df["sma"] = df["TP"].rolling(self.config_ndays).mean()
            df["mad"] = df["TP"].rolling(self.config_ndays).apply(lambda x: pd.Series(x).mad())
            df["CCI"] = (df["TP"] - df["sma"]) / (0.015 * df["mad"])
            
            df["pre1day_cci"] = df["CCI"].shift(1)
            df["trade"] = None
            df["trade"] = np.where((df["CCI"] >= -100) & (df["pre1day_cci"] < -100), "buy", df["trade"])
            df["trade"] = np.where((df["CCI"] <= 100) & (df["pre1day_cci"] > 100), "sell", df["trade"])
            
            df["total_buy_price"] = 0
            df["shares"] = 0
            df["buy_price"] = 0
            df["sell_price"] = 0
            df["revenue"] = 0
            df["rate"] = 0

            buy_price_list = []
            for i, x in df.iterrows():
                if x["trade"] == "buy":
                    buy_price_list.append(x["Close"])
                    df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                    df.loc[i, "shares"] = len(buy_price_list)
                    df.loc[i, "buy_price"] = np.mean(buy_price_list)
                elif x["trade"] == None:
                    if i > 0:
                        df.loc[i, "shares"] = df.loc[i-1, "shares"]
                        df.loc[i, "buy_price"] = df.loc[i-1, "buy_price"]
                        df.loc[i, "total_buy_price"] = df.loc[i-1, "total_buy_price"]
                elif x["trade"] == "sell" and df.loc[i-1, "shares"] > 0:
                    first_price = buy_price_list.pop(0)
                    df.loc[i, "revenue"] = df.loc[i, "Close"] - first_price
                    df.loc[i, "rate"] = df.loc[i, "revenue"] / first_price * 100
                    df.loc[i, "sell_price"] = df.loc[i, "Close"]
                    df.loc[i, "shares"] = len(buy_price_list)
                    if len(buy_price_list) > 0:
                        df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                        df.loc[i, "buy_price"] = np.mean(buy_price_list)
                    else:
                        df.loc[i, "total_buy_price"] = 0
                        df.loc[i, "buy_price"] = 0
            
            r_dict = {
                "symbol" : df.iloc[-1]["Symbol"],
                "date" : df.iloc[-1]["Date"],
                "trade_signal" : df.iloc[-1]["trade"],
                "price" : df.iloc[-1]["Close"]
            }
            return r_dict
            
        except Exception as e:
            # self._post_message(e)
            print(e)
            pass

    def _post_signal_to_slack(self):
        df = pd.DataFrame()
        buy_list = []
        sell_list = []
        idata= []

        if self.market == "kospi":
            for symbol in self.market_stock_dict[self.market]:
                try:
                    df = fdr.DataReader(symbol, start=self.start_date)
                    df["Symbol"] = symbol

                    r_dict = self._make_signal(df)

                    if r_dict["trade_signal"] == "buy":
                        buy_list.append(self.market_dict.get(r_dict["symbol"]))
                        print(r_dict)
                        idata.append(r_dict)
                    elif r_dict["trade_signal"] == "sell":
                        sell_list.append(self.market_dict.get(r_dict["symbol"]))
                        print(r_dict)
                        idata.append(r_dict)
                except Exception as e:
                    # self._post_message(e)
                    print(e)
                    pass

        elif self.market == "sp500":
            for symbol in self.market_stock_dict[self.market]:
                try:
                    df = yf.download(symbol, start=self.start_date, show_errors=False)
                    df["Symbol"] = symbol
                    
                    r_dict = self._make_signal(df)

                    if r_dict["trade_signal"] == "buy":

                        buy_list.append(r_dict["symbol"])
                        print(r_dict)
                        idata.append(r_dict)
                    elif r_dict["trade_signal"] == "sell":
                        sell_list.append(r_dict["symbol"])
                        print(r_dict)
                        idata.append(r_dict)

                except Exception as e:
                    # self._post_message(e)
                    print(e)
                    pass
            
        # 구매 데이터
        if len(buy_list) == 0:
            self._post_message("Today not exists buy stocks")
        if len(buy_list) > 0:
            self._post_message("Buy_stocks : %s"%(buy_list))

        # 판매 데이터
        if len(sell_list) == 0:
            self._post_message("Today not exists sell stocks")
        if len(sell_list) > 0:
            self._post_message("Sell_stocks : %s"%(sell_list))
        
        try:
            df = pd.DataFrame.from_dict(idata)
        except Exception as e:
            self._post_message(e)
            pass

        return df
        
    def _get_priority_house_jungso(self):
        driver = webdriver.Chrome(ChromeDriverManager().install())
        url = "https://www.smes.go.kr/sanhakin/websquare/wq_main.do"
        driver.get(url)
        driver.implicitly_wait(10)
        driver.find_element(By.XPATH, '//*[@id="genTopMenu_2_liTopMenu"]').click()
        driver.implicitly_wait(1)
        driver.find_element(By.XPATH, '//*[@id="genLeftMenu_3_leftMenuGrp"]').click()
        driver.implicitly_wait(1)

        idata = []
        # pagelist1_page_2
        search_date = datetime.today().date()
        str_search_date = datetime.strftime(search_date, "%Y-%m-%d")
        for page in range(1, 3):
            # print(page)
            driver.find_element(By.XPATH, '//*[@id="pagelist1_page_%s"]'%page).click()

            driver.implicitly_wait(1)
            for table_order in range (0, 10):
                title = driver.find_element_by_css_selector('#gridView1_cell_%s_0'%table_order).text
                str_start_date = driver.find_element_by_css_selector('#gridView1_cell_%s_3'%table_order).text.split(' ')[0]
                start_date = datetime.strptime(str_start_date, "%Y-%m-%d").date()
                if search_date <= start_date:
                    idict = {
                        "search_date" : str_search_date,
                        "title" : title,
                        "start_date" : str_start_date
                        }
                    idata.append(idict)
            if search_date > datetime.strptime(driver.find_element_by_css_selector('#gridView1_cell_9_3').text.split(' ')[0], "%Y-%m-%d").date():
                print(datetime.strptime(driver.find_element_by_css_selector('#gridView1_cell_9_3').text.split(' ')[0], "%Y-%m-%d").date())
                break
        driver.quit()
        if len(idata) > 0:
            text = "유효한 새로운 중소기업 장기근속자 주택 특별공급 없음"
            _post_message(self, text)
        else:
            text = idata
            _post_message(self, text)
        return idata

    def work(self):
        if __name__ == "__main__":
            self._get_stock_list()
        return self.market_stock_dict


In [ ]:
market_stock_dict = BaseService().work()

In [10]:
blocks = [
        {
            "type": "section",
            "text": {
                "type" : "mrkdwn",
                "text" : "마크다운 형식의 텍스트 테스트 메시지 발송"
            }
        }
    ]

In [11]:
BaseService()._post_message_with_slack_sdk(blocks=blocks)

C:\Users\senti\AppData\Local\Programs\Python\Python36\lib\site-packages\slack_sdk\web\internal_utils.py:271: UserWarning: The `text` argument is missing in the request payload for a chat.postMessage call - It's a best practice to always provide a `text` argument when posting a message. The `text` argument is used in places where content cannot be rendered such as: system push notifications, assistive technology such as screen readers, etc.
  warnings.warn(message, UserWarning)


{'ok': True, 'channel': 'C02HFSLTCGZ', 'ts': '1650527236.806379', 'message': {'bot_id': 'B02MBPXL74Z', 'type': 'message', 'text': "This content can't be displayed.", 'user': 'U02MNS36XU0', 'ts': '1650527236.806379', 'app_id': 'A02MBPWAH7B', 'team': 'T9G6QKR96', 'bot_profile': {'id': 'B02MBPXL74Z', 'app_id': 'A02MBPWAH7B', 'name': 'StockTrader', 'icons': {'image_36': 'https://a.slack-edge.com/80588/img/plugins/app/bot_36.png', 'image_48': 'https://a.slack-edge.com/80588/img/plugins/app/bot_48.png', 'image_72': 'https://a.slack-edge.com/80588/img/plugins/app/service_72.png'}, 'deleted': False, 'updated': 1650524077, 'team_id': 'T9G6QKR96'}, 'blocks': [{'type': 'section', 'block_id': 'dX2', 'text': {'type': 'mrkdwn', 'text': '마크다운 형식의 텍스트 테스트 메시지 발송', 'verbatim': False}}]}}


In [ ]:
ilist = []
for stock in market_stock_dict["kospi"]:
    symbol = stock+".KS"
    ilist.append(yf.Ticker(symbol).info)
market_info_df = pd.DataFrame(ilist)
market_info_df.head()

In [ ]:
kospi_df = pd.DataFrame(ilist)
kospi_df.head()

In [ ]:
kospi_df['sharesOutstanding']

In [ ]:
import yfinance as yf
total_df = pd.DataFrame()
for symbol in list(market_stock_dict["kospi"].keys()):
    df = yf.download(symbol+".KS", start=BaseService().start_date, auto_adjust = True, action=True, threads=False).reset_index()
    # y_df["Symbol"] = symbol
    # signal_dict, df =_make_cci_signal(BaseService(), y_df)
    # total_df = total_df.append(df)
    break

df.head()

In [ ]:
df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3
df["sma"] = df["TP"].rolling(20).mean()
df["mad"] = df["TP"].rolling(20).apply(lambda x: pd.Series(x).mad())
df["CCI"] = (df["TP"] - df["sma"]) / (0.015 * df["mad"])
df["pre1day_CCI"] = df["CCI"].shift(1)

In [ ]:
yf.download("282330.KS", start=str_start_date, auto_adjust=True).reset_index()

In [ ]:
financial_engineering_days = 252
end_date = datetime.today().date()
str_end_date = datetime.strftime(end_date, "%Y-%m-%d")
start_date = (end_date - timedelta(days=financial_engineering_days))
str_start_date = datetime.strftime(start_date, "%Y-%m-%d")
idata = []
for stock in market_stock_dict["kospi"]:
    df = pd.DataFrame()
    symbol = stock+".KS"
    print(symbol)
    datetime.today().date()
    try:
        df = yf.Ticker(symbol).history(interval="1d", start = str_start_date, end = str_end_date, auto_adjust=True).reset_index()
        df = df[df["Volume"]>0].reset_index(drop=True)
        config_ndays = 20
        
        # 수익률 계산하기
        df["daily_rtn"] = df["Close"].pct_change()  # 퍼센트 변화율
        df["st_rtn"] = (1 + df["daily_rtn"]).cumprod()  # 누적곱 계산함수 return cumulative product over a DataFrame or Series axis.
        gagr = (df.iloc[-1]['st_rtn'] ** (financial_engineering_days / len(df.index)) - 1) * 100

        df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3
        df["sma"] = df["TP"].rolling(config_ndays).mean()
        df["mad"] = df["TP"].rolling(config_ndays).apply(lambda x: pd.Series(x).mad())
        df["CCI"] = (df["TP"] - df["sma"]) / (0.015 * df["mad"])

        df["pre1day_cci"] = df["CCI"].shift(1)
        df["trade"] = None
        df["trade"] = np.where((df["CCI"] >= -100) & (df["pre1day_cci"] < -100), "buy", df["trade"])
        df["trade"] = np.where((df["CCI"] <= 100) & (df["pre1day_cci"] > 100), "sell", df["trade"])

        df["total_buy_price"] = 0
        df["shares"] = 0
        df["buy_price"] = 0
        df["sell_price"] = 0
        df["revenue"] = 0
        df["rate"] = 0

        buy_price_list = []
        for i, x in df.iterrows():
            if x["trade"] == "buy":
                buy_price_list.append(x["Close"])
                df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                df.loc[i, "shares"] = len(buy_price_list)
                df.loc[i, "buy_price"] = np.mean(buy_price_list)
            elif x["trade"] == None:
                if i > 0:
                    df.loc[i, "shares"] = df.loc[i-1, "shares"]
                    df.loc[i, "buy_price"] = df.loc[i-1, "buy_price"]
                    df.loc[i, "total_buy_price"] = df.loc[i-1, "total_buy_price"]
            elif x["trade"] == "sell" and df.loc[i-1, "shares"] > 0:
                first_price = buy_price_list.pop(0)
                df.loc[i, "revenue"] = df.loc[i, "Close"] - first_price
                df.loc[i, "rate"] = df.loc[i, "revenue"] / first_price * 100
                df.loc[i, "sell_price"] = df.loc[i, "Close"]
                df.loc[i, "shares"] = len(buy_price_list)
                if len(buy_price_list) > 0:
                    df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                    df.loc[i, "buy_price"] = np.mean(buy_price_list)
                else:
                    df.loc[i, "total_buy_price"] = 0
                    df.loc[i, "buy_price"] = 0

        r_dict = {
            "symbol" : symbol,
            "date" : datetime.strftime(df.iloc[-1]["Date"], "%Y-%m-%d"),
            "trade_signal" : df.iloc[-1]["trade"],
            "price" : df.iloc[-1]["Close"],
            "remain_shares" : df.iloc[-1]["shares"],
            "holding_shares_buy_price" : df.iloc[-1]["buy_price"],
            "cci_rtn" : df[df["trade"] == "sell"].iloc[-1]["rate"],
            "buy_and_hold_rtn" : gagr
        }
        idata.append(r_dict)
    except Exception as e:
        print(e)
        pass
    print(r_dict)

In [ ]:
df.head(40)

In [ ]:
df["buy_price"] = None
for i, x in df.iterrows():
    df.loc[i, "buy_price"] = 
    break

In [ ]:
print(round(total_df[total_df["trade"]=="Buy"].groupby(["Date"])["Symbol"].nunique().reset_index(name="buy_count")["buy_count"].mean(), 2))
print(round(total_df[total_df["trade"]=="Sell"].groupby(["Date"])["Symbol"].nunique().reset_index(name="sell_count")["sell_count"].mean(), 2))

In [ ]:
total_df["Date"].iloc[-1]

In [ ]:
total_df[total_df["trade"]=="Buy"].sort_values("Date")

In [ ]:
buy_df = total_df[total_df["trade"] == "Buy"].sort_values("Date").reset_index(drop=True)
buy_df[buy_df["Date"]==buy_df["Date"].iloc[-1]]

In [ ]:
print(total_df[(total_df["trade"]=="Buy") & (total_df["Date"] == total_df["Date"].iloc[-1])]["Symbol"].nunique())
total_df[(total_df["trade"]=="Sell") & (total_df["Date"] == total_df["Date"].iloc[-1])]["Symbol"].nunique()

In [ ]:
print(round(total_df[total_df["trade"]=="Buy"].iloc[-1]["Symbol"].nunique(), 2))
print(round(total_df[total_df["trade"]=="Sell"].iloc[-1]["Symbol"], 2))

In [ ]:
df, buy_stock_list, sell_stock_list = _collect_cci_signal(BaseService(), market_stock_dict)
df.head()

In [ ]:
buy_df = df[df["name"].isin(buy_stock_list)]
sell_df = df[df["name"].isin(sell_stock_list)]
stock_info_list = []

for i, x in buy_df.iterrows():
    candidate_dict = {
        "type" : "section",
        "text" : {
            "type" : "mrkdwn",
            "text" : f'매매신호 : {x["trade_signal"]}\n\
이름 : {x["name"]}\n\
symbol : {x["symbol"]}\n\
매매신호가격 : {x["signal_price"]}\n\
CCI수익률-구매후보유수익률 : {x["cci_buy_and_hold_diff_rtn"]}\n\
{BaseService().start_date}CCI수익률 : {x["cci_rtn"]}\n\
{BaseService().start_date}구매후보유수익률 : {x["buy_and_hold_rtn"]}\n\
{BaseService().target_days}일 이내 평균최고수익률 : {x["max_rtn"]}\n\
{BaseService().target_days}일 이내 구매 후 평균최고수익일수 : {x["max_rtn_day"]}\n\
CCI전략수행시 보유주식수 : {x["remain_holding_shares"]}\n\
CCI전략수행시 보유주식 평균 구매가격: {x["holding_shares_buy_price"]}'
}
}
    stock_info_list.append(candidate_dict)

for i, x in sell_df.iterrows():
    candidate_dict = {
        "type" : "section",
        "text" : {
            "type" : "mrkdwn",
            "text" : f'매매신호 : {x["trade_signal"]}\n\
이름 : {x["name"]}\n\
symbol : {x["symbol"]}\n\
매매신호가격 : {x["signal_price"]}\n\
CCI수익률-구매후보유수익률 : {x["cci_buy_and_hold_diff_rtn"]}\n\
{BaseService().start_date}CCI수익률 : {x["cci_rtn"]}\n\
{BaseService().start_date}구매후보유수익률 : {x["buy_and_hold_rtn"]}\n\
{BaseService().target_days}일 이내 평균최고수익률 : {x["max_rtn"]}\n\
{BaseService().target_days}일 이내 구매 후 평균최고수익일수 : {x["max_rtn_day"]}\n\
CCI전략수행시 보유주식수 : {x["remain_holding_shares"]}\n\
CCI전략수행시 보유주식 평균 구매가격: {x["holding_shares_buy_price"]}'
}
}
    stock_info_list.append(candidate_dict)
print(stock_info_list)

In [ ]:
_post_message_with_slack_sdk(BaseService(), blocks=stock_info_list)

In [ ]:
buy_stock_list = df[df["trade_signal"]=="Buy"].sort_values(["cci_buy_and_hold_diff_rtn", f"within_{BaseService().target_days}days_expected_rtn"], ascending=False)["name"].tolist()[:BaseService().max_trade_stock_count]
buy_stock_list

In [ ]:
sell_stock_list = df[df["trade_signal"]=="Sell"].sort_values(["cci_buy_and_hold_diff_rtn", f"within_{BaseService().target_days}days_expected_rtn"])["name"].tolist()[:BaseService().max_trade_stock_count]
sell_stock_list

In [ ]:
df[df["name"].isin(buy_stock_list)]

In [ ]:
t_list = range(1, 6)
t_list

In [ ]:
df[df["trade_signal"]=="Sell"].sort_values(["cci_buy_and_hold_diff_rtn", "within_7days_expected_rtn"])

In [ ]:
df[df["trade_signal"]=="Buy"].sort_values(["cci_buy_and_hold_diff_rtn", "within_7days_expected_rtn"], ascending=False)

In [ ]:
r_df[["after_1day_price", "Close"]]

In [ ]:
df[~df["trade"].isnull()].head(20)

In [ ]:
df.info()

In [ ]:
def _get_kr_stock_info():
    r_data = []
    params = {
        "ServiceKey" : apisdata["getstockinfo"]["decoding_key"],
        "pageNo" : 1,
        "numOfRows" : 9999,
        "resultType" : "json"
        }
    url = apisdata["getstockinfo"]["request_url"]
    response = requests.get(url, params=params)
    if response.ok:
        get_dict = json.loads(response.text)
        r_data = get_dict["response"]["body"]["items"]["item"]

    r_df = pd.DataFrame(r_data)

    return r_df
    

In [ ]:
df = _get_kr_stock_info()
# 상장폐지(lstgAbolDt) 안되었고, 예탁취소(dpsgCanDt)도 안된 종목만 본다.
f_df = df[(df["lstgAbolDt"] == '') & (df["dpsgCanDt"] == '')]

# 보통주
common_df = f_df[f_df["scrsItmsKcdNm"]=="보통주"]
common_df


In [ ]:
df.head()

In [ ]:
df

In [ ]:
df[df["stckIssuCmpyNm"].str.contains("삼성")]

In [ ]:
kospi_tickers = fdr.StockListing("KOSPI")["Symbol"].tolist()
kospi_tickers

In [ ]:
for ticker in kospi_tickers:
    df = fdr.DataReader(ticker)
    df.head()
    break

In [ ]:
df

In [ ]:
datetime.strptime("09:00:00", "%H:%M:%S").time()

In [ ]:
datetime.combine((datetime.today() + timedelta(days=1)), datetime.strptime("09:00", "%H:%M").time())

In [ ]:
datetime.now().time().hour

In [ ]:
if datetime.now().time() < time(hour=15, minute=9):
    print("aaa")